In [1]:
from NCHelper import NCHelper
from datetime import datetime, timedelta
import xarray as xr
import numpy as np
import rasterio
from rasterio.crs import CRS
import time
import os
import shutil

In [2]:
date_str = "2023110800"
ds = f"http://182.16.248.173:8080/dods/INA-NWP/{date_str}/{date_str}-d01-asim"
# ds = "sample\indonesia-20230901-20230907.nc"

nch = NCHelper(ds)

reading dataset...
reading dataset done


c:\Users\falbas\miniconda3\Lib\site-packages\xarray\coding\times.py:167: SerializationWarning: Ambiguous reference date string: 1-1-1 00:00:0.0. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0001-1-1 00:00:0.0). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)


In [16]:
nch.ds['xlat'][0][0][-1][-1].values

array(9.897057, dtype=float32)

In [15]:
nch.ds['xlong'][0][0][0][-1].values

array(143.89561, dtype=float32)

In [18]:
lat = nch.ds['lat']
lon = nch.ds['lon']
u = nch.ds['u']
v = nch.ds['v']
tc = nch.ds['tc']

In [47]:
llat = []
for i in lon.values:
  llat.append(lat.values)
llat = np.array(llat)
llat = np.rot90(llat, k=3)

In [49]:
llat[-1][-1]

-15.0001297

In [50]:
output_name = 'lat'
# var = nch.ds[output_name]
var = llat
var = np.flip(var, axis=0)
transform = rasterio.transform.from_bounds(west=lon[0],south=lat[0],east=lon[-1],north=lat[-1],width=lon.shape[0],height=lat.shape[0])
with rasterio.open(
    f"{output_name}.tif",
    "w",
    driver="GTiff",
    height=lat.shape[0],
    width=lon.shape[0],
    count=1,
    dtype=var.dtype,
    crs="+proj=latlong",
    transform=transform,
    nodata=-9999
) as dst:
    dst.write(var, 1)

In [19]:
nch.nc2tif(nch.ds['xlat'][0][0].values, 'xlat', lat.values, lon.values)

In [8]:
nch.nc2rgbtif('tc1', 'tc_color.txt')
nch.nc2rgbtif('tc2', 'tc_color.txt')

generating tc1 rgb geotif images...
output: d:/Kuliah/MBKM/PKKM BMKG/NCHelper/tc1_rgb.tif
generating tc2 rgb geotif images...
output: d:/Kuliah/MBKM/PKKM BMKG/NCHelper/tc2_rgb.tif
